# H3 Boundary Coverage Demo

This notebook demonstrates how to cover the **boundary** (perimeter) of an arbitrary polygon with H3 cells at a specific resolution.

This is useful when you want to trace a shape without filling its interior, which is much more efficient for high resolutions (like Res 15).

In [7]:
import h3
import folium
from shapely.geometry import Polygon, mapping
import h3_toolkit  # Using toolkit for visualization helpers if needed, though mostly standard h3 here

# 1. Define a Sample Polygon (San Francisco Area)
# format: list of [lng, lat]
polygon_coords = [
    [-122.42, 37.77], 
    [-122.41, 37.77], 
    [-122.41, 37.76], 
    [-122.42, 37.76], 
    [-122.42, 37.77]  # Close the loop
]

sample_polygon = {
    "type": "Polygon",
    "coordinates": [polygon_coords]
}

# Center for map
center_lat, center_lng = 37.765, -122.415

## Algorithm: "Walk the Line"

To trace the boundary:
1. Iterate through every pair of vertices $(p_i, p_{i+1})$.
2. Find the H3 cell for each vertex.
3. Use `h3.grid_path_cells()` to find the line of hexagons connecting them.
4. Union all these paths.

In [8]:
def get_boundary_cells(polygon_geojson, res):
    # 1. Get the list of coordinates (the ring)
    coords = polygon_geojson['coordinates'][0]
    
    boundary_cells = set()
    
    # 2. Walk through every pair of points
    for i in range(len(coords) - 1):
        p1 = coords[i]      # [lng, lat]
        p2 = coords[i+1]    # [lng, lat]
        
        # Get the cell at each vertex
        # Note: h3.latlng_to_cell expects (lat, lng)
        c1 = h3.latlng_to_cell(p1[1], p1[0], res)
        c2 = h3.latlng_to_cell(p2[1], p2[0], res)
        
        # 3. Connect them with a line of cells
        try:
            path = h3.grid_path_cells(c1, c2)
            boundary_cells.update(path)
        except Exception as e:
            # Fallback for very long distances or edge cases
            print(f"Warning: could not compute path between {c1} and {c2}: {e}")
            boundary_cells.add(c1)
            boundary_cells.add(c2)
            
    return boundary_cells

## Visualize the Result

We use **Resolution 11** for this demo so you can clearly see the cells on the map. 
(Resolution 15 is valid but extremely small, making it hard to see without zooming in very far).

In [9]:
# 1. Calculate Boundary Cells at Res 11
target_res = 11
boundary_cells = get_boundary_cells(sample_polygon, target_res)
print(f"Found {len(boundary_cells)} cells covering the boundary at Res {target_res}")

# 2. Visualize
m = folium.Map(location=[center_lat, center_lng], zoom_start=15, tiles='CartoDB positron')

# Draw the original Polygon (Blue Outline)
folium.GeoJson(
    sample_polygon,
    style_function=lambda x: {'color': 'blue', 'fillOpacity': 0.1, 'weight': 2},
    name="Target Polygon"
).add_to(m)

# Draw the Boundary Cells (Red)
# We convert each cell to its boundary polygon for display
for cell in boundary_cells:
    boundary = h3.cell_to_boundary(cell)
    # Swap (lat, lng) to (lng, lat)
    boundary_lnglat = [[(p[1], p[0]) for p in boundary]]
    # Close the loop
    boundary_lnglat[0].append(boundary_lnglat[0][0])
    poly = {"type": "Polygon", "coordinates": boundary_lnglat}
    feature = {"type": "Feature", "geometry": poly, "properties": {}}
    folium.GeoJson(
        feature,
        style_function=lambda x: {'color': 'red', 'fillOpacity': 0.6, 'weight': 0}
    ).add_to(m)

m

Found 85 cells covering the boundary at Res 11


## "Real Shape" Visualization
Using `h3_toolkit` to visualize the merged boundary of these cells.
This creates a single "jagged ring" polygon.

In [10]:
# Convert the set of cells into a single merged MultiPolygon
from shapely.geometry import shape, mapping
from shapely.ops import unary_union

    # Convert to shapely Polygons (swap lat,lng to lng,lat)
    polys = []
    for c in boundary_cells:
        boundary = h3.cell_to_boundary(c)
        boundary_lnglat = [(p[1], p[0]) for p in boundary]
        polys.append(Polygon(boundary_lnglat))
merged_boundary = unary_union(polys)

m2 = folium.Map(location=[center_lat, center_lng], zoom_start=15, tiles='CartoDB positron')

# Original
folium.GeoJson(
    sample_polygon,
    style_function=lambda x: {'color': 'blue', 'fillOpacity': 0.1, 'weight': 1}
).add_to(m2)

# Merged Boundary Trace (Orange)
folium.GeoJson(
    mapping(merged_boundary),
    style_function=lambda x: {'color': 'orange', 'fillOpacity': 0.6, 'weight': 2}
).add_to(m2)

m2

IndentationError: unexpected indent (3566987281.py, line 6)